In [ ]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import numpy as np
import sqlite3
import urllib
import urllib.request
import re
import datetime
import logging
import time
from selenium import webdriver

In [ ]:
now = datetime.datetime.now()
today = str(now.date())

In [ ]:
logging.basicConfig(filename='DA' + today + '.log', level=logging.DEBUG, format='%(asctime)s:%(name)s:%(message)s')

In [ ]:
url = 'https://job.jobnet.dk/CV/FindWork?Offset=0&SortValue=BestMatch&OccupationAreas=It%2520og%2520teleteknik'

# create a new Firefox session
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)
        

In [ ]:
#decleare empty dataframe to store everything related to one role
role_dataframe_all = pd.DataFrame()
page_number = 0

In [ ]:
#iterate over job type list with dynamic url
#for each job iterate through all page to get all positions and store in dataframe, store: position name, company name, location
print(datetime.datetime.now())

try:

    while page_number < 680:
        
        print('waiting 5 sec')
        time.sleep(5)
    
        print(page_number)
        #print('connecting to : {}'.format(str(page_number)), datetime.datetime.now())
        
        soup = BeautifulSoup(driver.page_source, 'lxml')

        job_title_results = soup.find_all('div', attrs={'class':'job-ad-summary tear-off'}) 
 

        #iterate over certain page to store position, company, and location in their own list

        pos_all = []
        for result in job_title_results:
            position = result.find('h2').text
            job_description_url = result.find('a').get('href')
            company = result.find('h3').text


            pos_all.append([position, job_description_url, company])

        #creating merged dataframe
        role_dataframe_page = pd.DataFrame(pos_all, columns=['position', 'url', 'company'])

        #adding data to role related dataframe
        role_dataframe_all = role_dataframe_all.append(role_dataframe_page, ignore_index=True)
        #increment page_number
        #print(now)
        #print(datetime.datetime.now())
        page_number += 20
        print('waiting 5 seconds')
        time.sleep(5)
        
        driver.get('https://job.jobnet.dk/CV/FindWork?Offset='+ str(page_number) + 
                   '&SortValue=BestMatch&OccupationAreas=It%2520og%2520teleteknik') 
        
        
except Exception as e:
    logging.exception("message")
    print("Error detected and logged")

In [ ]:
role_dataframe_all.drop_duplicates(inplace=True)

In [ ]:
#role_dataframe_all.to_excel('DA_IT_' + today+ '.xlsx')

In [ ]:
role_dataframe_all = pd.read_excel('DA_IT_2018-10-31.xlsx')

In [ ]:
role_dataframe_all = role_dataframe_all[role_dataframe_all['url'].str.contains('jobnet')]

In [ ]:
#role_dataframe_all = role_dataframe_all.sample(n=500)

In [ ]:
driver.close()

In [ ]:
#role_dataframe_all = role_dataframe_all[0:3]

In [ ]:
url = 'https://job.jobnet.dk/CV/FindWork/Details/4898853'

# create a new Firefox session
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

In [ ]:
def description_extractor(url):
    try:
        print(url)
        driver.get(url)
        
        print('wait 5 seconds')
        time.sleep(5)
        
        soup = BeautifulSoup(driver.page_source, 'lxml')

        job_desc_results = soup.find('div', attrs={'class':'ng-binding ng-scope'}).text

        job_desc_results = re.sub('\n', '', job_desc_results.strip())

        job_location = soup.find_all('ul', attrs={'class':'job-info-list'})
        job_location = job_location[3].text + job_location[4].text
        
        job_location = re.sub('\n', ' ', job_location.strip())
        
        
        print('wait 5 sec')
        time.sleep(5)
        return [job_desc_results, job_location]
    except Exception as e:
        logging.exception("message")
        print("Error detected and logged")

In [ ]:
all = []

In [ ]:
all.append(role_dataframe_all['url'].apply(description_extractor))
all = pd.DataFrame(all).T
all['url'] = all['url'].apply(lambda text: ['mv-9', 'mv-9'] if text is None else text)
all = all.values.tolist()
flat_list = [item for sublist in all for item in sublist]
all_df = pd.DataFrame(flat_list, columns=['description', 'location'])

In [ ]:
role_dataframe_all = pd.merge(role_dataframe_all, all_df, left_index=True, right_index=True)

In [ ]:
role_dataframe_all.to_excel('DA_FULL_2018_11_01.xlsx')